In [16]:
import pdfplumber
import os

In [2]:
input_dir = "../output/pdf_download_list/pdfs/"
files = os.listdir(input_dir)
court_summaries_commonpleas = [f for f in files if 'CS_CP' in f]
court_summaries_magisterial = [f for f in files if 'CS_MJ' in f]

In [25]:
counties = [
            "Adams", "Allegheny", "Armstrong", "Beaver", "Bedford", "Berks",
            "Blair", "Bradford", "Bucks", "Butler", "Cambria", "Cameron",
            "Carbon", "Centre", "Chester", "Clarion", "Clearfield", "Clinton",
            "Columbia", "Crawford", "Cumberland", "Dauphin", "Delaware", "Elk",
            "Erie", "Fayette", "Forest", "Franklin", "Fulton", "Greene",
            "Huntingdon", "Indiana", "Jefferson", "Juniata", "Lackawanna",
            "Lancaster", "Lawrence", "Lebanon", "Lehigh", "Luzerne", "Lycoming",
            "McKean", "Mercer", "Mifflin", "Monroe", "Montgomery", "Montour",
            "Northampton", "Northumberla", "Perry", "Philadelphia", "Pike",
            "Potter", "Schuylkill", "Snyder", "Somerset", "Sullivan",
            "Susquehanna", "Tioga", "Union", "Venango", "Warren", "Washington",
            "Wayne", "Westmoreland", "Wyoming", "York"
            ]

In [ ]:
pdf_path = "../output/pdf_download_list/pdfs/CS_CP-51-CR-0013145-2010.pdf"
pdf = pdfplumber.open(pdf_path)
pages = pdf.pages
cs_dict = {}

for page in pages:
    text = page.extract_text().split("\n")
    
    # Beginning part of every court summary in court of common pleas has a block of text with person information.
    poi_start_index = [i for i, x in enumerate(text) if "DOB:" in x][0]
    poi_end_index = [i for i,x in enumerate(text) if "closed" in x.lower() or "inactive" in x.lower() or "active" in x.lower()][0]
    poi = text[poi_start_index:poi_end_index]

    # Name, DOB, and Sex appear on the first line.
    cs_dict["name"] = poi[0].split("DOB:")[0].strip()
    cs_dict["dob"] = poi[0].split("DOB:")[1].split("Sex:")[0].strip()
    cs_dict["sex"] = poi[0].split("DOB:")[1].split("Sex:")[1].strip()

    # Location and Eye Color appear on the second line.
    cs_dict["location"] = poi[1].split("Eyes:")[0].strip()
    cs_dict["eyes"] = poi[1].split("Eyes:")[1].strip()

    # Alias and hair color are on the third line, but alias is blank on this line.
    cs_dict["hair"] = poi[2].split("Hair:")[1].strip()

    # The first alias and race are on the fourth line.
    alias = poi[3].split("Race:")[0].strip()
    cs_dict["race"] = poi[3].split("Race:")[1].strip()                          

    # The rest of the aliases are on subequent lines.
    remainder_alias = poi[4:len(poi)]
    remainder_alias.append(alias)
    cs_dict["alias"] = remainder_alias

    
    print("Name: " + cs_dict["name"])
    print("DOB: " + cs_dict["dob"])
    print("Sex: " + cs_dict["sex"])
    print("Location: " + cs_dict["location"])
    print("Eyes: " + cs_dict["eyes"])
    print("Hair: " + cs_dict["hair"])
    print("Race: " + cs_dict["race"])
    print(cs_dict["alias"])

    cases_to_be_parsed = True
    current_line_index = poi_end_index
    case_nr = 0
    case_nr_idx = "case_" + str(case_nr)
    seq_nr = 0
    seq_nr_idx = "seq_" + str(seq_nr)
    sentence_nr = 0
    sentence_nr_idx = "sentence_" + str(sentence_nr)

    # Loop through all the cases.
    while(current_line_index < len(text)):
        cur_line = text[current_line_index]

        # Check if the current line is a new set of case (statuses).
        if(cur_line.lower() in "closed" or cur_line.lower() in "inactive" or cur_line.lower() in "active"):
            case_status = cur_line.lower()
            cs_dict[case_status] = {}
            cs_dict[case_status][case_nr_idx] = {}
        # Check if the current line is a new county.
        elif(cur_line in counties):
            cs_dict[case_status][case_nr_idx]["county"] = cur_line.lower()
        # If we are not on a new county or new case status, the top three lines of every case follow a consistent pattern. It also signals a new case.
        # 1st line: Docket Number, Proc. Status, DC Number, and OTN Number.
        # 2nd line: Arrest date, disposition date, and disposition judge.
        # 3rd line: Defense attorney
        elif("Proc Status: " in cur_line):

            # HOW TO FIX THIS WHEN LOOPING THROUGH FOR THE FIRST TIME. ADD A FLAG TO INDICATE IT IS THE FIRST CASE? MIGHT BE TOO COMPLICATED.
            cs_dict[case_status][case_nr_idx]["docket_number"] = cur_line.split("Proc Status:")[0].strip()
            cs_dict[case_status][case_nr_idx]["proc_status"] = cur_line.split("Proc Status:")[1].split("DC No:")[0].strip().lower()
            cs_dict[case_status][case_nr_idx]["dc_nr"] = cur_line.split("Proc Status:")[1].split("DC No:")[1].split("OTN:")[0].strip().lower()
            cs_dict[case_status][case_nr_idx]["otn_nr"] = cur_line.split("Proc Status:")[1].split("DC No:")[1].split("OTN:")[1].strip().lower()

        current_line_index += 1
    
    print(cs_dict)

Name: Johnson, Edward
DOB: 09/28/1988
Sex: Male
Location: Philadelphia, PA 19141
Eyes: Brown
Hair: Black
Race: Black
['Edward Joshua Johnson', 'Edward S. Johnson', 'Edward Johnson']


KeyError: 'case_-1'

In [12]:
test = "name"
test_dict = {}
test_dict[test] = "monkey"


dict = {}
dict["closed"] = {}
dict["closed"]["case_0"] = {}
dict["closed"]["case_0"]["location"] = "Philadelphia"
dict["closed"]["case_0"]["docket_nr"] = "CP-51-CR-0013145-2010"
dict["closed"]["case_0"]["proc_status"] = "completed"
dict["closed"]["case_0"]["dc_nr"] = "10240"
dict["closed"]["case_0"]["OTN"] = "N 700005-5"
dict["closed"]["case_0"]["arrest_date"] = "08/31/2010"
dict["closed"]["case_0"]["disp_date"] = "03/23/2011"
dict["closed"]["case_0"]["disp_judge"] = "Paula A. Patrick"
dict["closed"]["case_0"]["def_attorney"] = "Edward C. Jr. Meehan"
dict["closed"]["case_0"]["seq_0"] = {}
dict["closed"]["case_0"]["seq_0"]["seq_num"] = "1"
dict["closed"]["case_0"]["seq_0"]["statute"] = "18&6106&&A1"
dict["closed"]["case_0"]["seq_0"]["grade"] = "F3"
dict["closed"]["case_0"]["seq_0"]["description"] = "firearms not to be carried"
dict["closed"]["case_0"]["seq_0"]["disposition"] = "guilty"
dict["closed"]["case_0"]["seq_0"]["sentence_0"] = {}
dict["closed"]["case_0"]["seq_0"]["sentence_0"]["sentence_date"] = "05/18/2011"
dict["closed"]["case_0"]["seq_0"]["sentence_0"]["sentence_type"] = "confinement"
dict["closed"]["case_0"]["seq_0"]["sentence_0"]["program_period"] = "11 1/2 - 23 months"
dict["closed"]["case_0"]["seq_0"]["sentence_0"]["sentence_length"] = "min: 11 months 15 days max: 23 months"
dict["closed"]["case_0"]["seq_0"]["sentence_1"] = {}
dict["closed"]["case_0"]["seq_0"]["sentence_1"]["sentence_date"] = "05/18/2011"
dict["closed"]["case_0"]["seq_0"]["sentence_1"]["sentence_type"] = "probation"
dict["closed"]["case_0"]["seq_0"]["sentence_1"]["program_period"] = "2 years"
dict["closed"]["case_0"]["seq_0"]["sentence_1"]["sentence_length"] = "max 2 years"
dict["closed"]["case_0"]["seq_1"] = {}
dict["closed"]["case_0"]["seq_1"]["seq_num"] = "2"
dict["closed"]["case_0"]["seq_1"]["statute"] = "18&6108"
dict["closed"]["case_0"]["seq_1"]["grade"] = "M1"
dict["closed"]["case_0"]["seq_1"]["description"] = "carry firearms in public"
dict["closed"]["case_0"]["seq_1"]["disposition"] = "guilty"

In [13]:
print(dict)
print(test_dict)

{'closed': {'case_0': {'location': 'Philadelphia', 'docket_nr': 'CP-51-CR-0013145-2010', 'proc_status': 'completed', 'dc_nr': '10240', 'OTN': 'N 700005-5', 'arrest_date': '08/31/2010', 'disp_date': '03/23/2011', 'disp_judge': 'Paula A. Patrick', 'def_attorney': 'Edward C. Jr. Meehan', 'seq_0': {'seq_num': '1', 'statute': '18&6106&&A1', 'grade': 'F3', 'description': 'firearms not to be carried', 'disposition': 'guilty', 'sentence_0': {'sentence_date': '05/18/2011', 'sentence_type': 'confinement', 'program_period': '11 1/2 - 23 months', 'sentence_length': 'min: 11 months 15 days max: 23 months'}, 'sentence_1': {'sentence_date': '05/18/2011', 'sentence_type': 'probation', 'program_period': '2 years', 'sentence_length': 'max 2 years'}}, 'seq_1': {'seq_num': '2', 'statute': '18&6108', 'grade': 'M1', 'description': 'carry firearms in public', 'disposition': 'guilty'}}}}
{'name': 'monkey'}
